# Informer model

In [1]:
#!pip install deepts_forecasting

### Import libraries

In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from deepts_forecasting.utils.data import TimeSeriesDataSet
from deepts_forecasting.utils.data.encoders import TorchNormalizer
from deepts_forecasting.datasets import AirPassengersDataset
from deepts_forecasting.models.informer.informer import Informer


D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataset

In [2]:
data = AirPassengersDataset().load()
data['year'] = data['Month'].dt.year
data['month'] = data['Month'].dt.month
data['group'] = '0'
data['time_idx'] = np.arange(len(data))
data['Passengers'] = data['Passengers'].astype(float)
data['month'] = data['month'].astype('str')
data.head()

,Month,Passengers,year,month,group,time_idx
0,1949-01-01,112.0,1949,1,0,0
1,1949-02-01,118.0,1949,2,0,1
2,1949-03-01,132.0,1949,3,0,2
3,1949-04-01,129.0,1949,4,0,3
4,1949-05-01,121.0,1949,5,0,4


### Split train/test sets

In [3]:
max_encoder_length = 18
max_prediction_length = 12

training_cutoff = data["time_idx"].max() - max_encoder_length - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    max_encoder_length= max_encoder_length,
    min_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    min_prediction_length=max_prediction_length,
    time_idx="time_idx",
    target="Passengers",
    group_ids=["group"],
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=['month'],
    time_varying_known_reals=[],
    time_varying_unknown_reals=["Passengers"],
    time_varying_unknown_categoricals=[],
    target_normalizer=TorchNormalizer(method="standard",
                                      transformation=None),
    )

training.get_parameters()
validation = TimeSeriesDataSet.from_dataset(training,
                                            data[lambda x: x.time_idx > training_cutoff])

batch_size = 16
train_dataloader = DataLoader(training, batch_size=batch_size, shuffle=False, drop_last=False)
val_dataloader = DataLoader(validation, batch_size=batch_size, shuffle=False, drop_last=False)


### Define model

In [4]:
pl.seed_everything(1234)
# create PyTorch Lighning Trainer with early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4,
                                    patience=60, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

trainer = pl.Trainer(
    max_epochs=300,
    gpus=0,  # run on CPU, if on multiple GPUs, use accelerator="ddp"
    gradient_clip_val=0.1,
    limit_train_batches=30,  # 30 batches per epoch
    callbacks=[lr_logger, early_stop_callback],
    logger=TensorBoardLogger("lightning_logs")
)

model = Informer.from_dataset(training,
                                  d_model=16,
                                  d_ff=32,
                                  n_heads=2,
                                  num_layers=2,
                                      )
model.summarize

Global seed set to 1234
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


<bound method LightningModule.summarize of Informer(
  (loss): MSELoss()
  (logging_metrics): ModuleList()
  (encoder_input_linear): Linear(in_features=7, out_features=16, bias=True)
  (decoder_input_linear): Linear(in_features=6, out_features=16, bias=True)
  (encoder_positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder_positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (informer_encoder): InformerEncoder(
    (encoder_layers): ModuleList(
      (0): InformerEncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): ProbAttention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (query_projection): Linear(in_features=16, out_features=16, bias=True)
          (key_projection): Linear(in_features=16, out_features=16, bias=True)
          (value_projection): Linear(in_features=16, out_features=16, bias=True)
          (out_projection): Linear(in_featur

In [5]:
model.hparams

"activation":                        relu
"categorical_groups":                {}
"d_ff":                              32
"d_model":                           16
"dropout":                           0.1
"embedding_labels":                  {'month': array(['1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype=object)}
"embedding_paddings":                []
"embedding_sizes":                   {'month': [12, 6]}
"learning_rate":                     0.001
"log_interval":                      -1
"log_val_interval":                  None
"logging_metrics":                   ModuleList()
"loss":                              L1Loss()
"max_encoder_length":                18
"max_prediction_length":             12
"monotone_constaints":               {}
"n_heads":                           2
"num_layers":                        2
"output_size":                       1
"output_transformer":                TorchNormalizer()
"static_categoricals":               []
"static_r

### Train model with early stopping

In [6]:
trainer.fit(
    model, train_dataloader=train_dataloader, val_dataloaders=val_dataloader,
)

# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = Informer.load_from_checkpoint(best_model_path)

# calcualte mean absolute error on validation set
actuals = torch.cat([model.transform_output(prediction=y, target_scale=x['target_scale'])
                     for x, y in iter(val_dataloader)])
predictions, x_index = best_model.predict(val_dataloader)
mae = (actuals - predictions).abs().mean()
# print('predictions shape is:', predictions.shape)
# print('actuals shape is:', actuals.shape)
print(torch.cat([actuals, predictions]))
print('MAE is:', mae)

D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\trainer.py:735: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(

  | Name                        | Type               | Params
-------------------------------------------------------------------
0 | loss                        | MSELoss            | 0     
1 | logging_metrics             | ModuleList         | 0     
2 | encoder_input_linear        | Linear             | 128   
3 | decoder_input_linear        | Linear             | 112   
4 | encoder_positional_encoding | PositionalEncoding | 0     
5 | decoder_positional_encoding | PositionalEncoding | 0     
6 | informer_encoder            | InformerEncoder    | 3.1 K 
7 | informer_decoder            | InformerDecoder    | 4.4 K 
8 | out_linear                  | Linear             | 17    
9 | embeddings 

D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 1234
D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\data_loading.py:432: UserWarning: The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_s

Epoch 0:  86%|██████████▎ | 6/7 [00:00<00:00, 33.05it/s, loss=1.53, v_num=20]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|█| 7/7 [00:00<00:00, 35.71it/s, loss=1.53, v_num=20, val_loss=9
Epoch 1:  86%|▊| 6/7 [00:00<00:00, 28.84it/s, loss=1.23, v_num=20, val_loss=9
Validating: 0it [00:00, ?it/s]
Epoch 1: 100%|█| 7/7 [00:00<00:00, 31.60it/s, loss=1.23, v_num=20, val_loss=7
Epoch 2:  86%|▊| 6/7 [00:00<00:00, 24.39it/s, loss=1.11, v_num=20, val_loss=7
Validating: 0it [00:00, ?it/s]
Epoch 2: 100%|█| 7/7 [00:00<00:00, 25.83it/s, loss=1.11, v_num=20, val_loss=7
Epoch 3:  86%|▊| 6/7 [00:00<00:00, 16.66it/s, loss=1.11, v_num=20, val_loss=7
Validating: 0it [00:00, ?it/s]
Epoch 3: 100%|█| 7/7 [00:00<00:00, 18.06it/s, loss=1.11, v_num=20, val_loss=7
Epoch 4:  86%|▊| 6/7 [00:00<00:00, 15.95it/s, loss=0.956, v_num=20, val_loss=
Validating: 0it [00:00, ?it/s]
Epoch 4: 100%|█| 7/7 [00:00<00:00, 17.07it/s, loss=0.956, v_num=20, val_loss=
Epoch 5:  86%|▊| 6/7 [00:00<00:00, 15.34it/s, loss=0.9, v_num=20,

TypeError: 'NoneType' object is not callable

TypeError: transform_output() missing 2 required positional arguments: 'prediction' and 'target_scale'

In [8]:
 for x, y in iter(val_dataloader):
    print(x['target_scale'],y)

tensor([[[237.7456,  88.2423]]], dtype=torch.float64) tensor([[[2.0314],
         [1.7367],
         [2.0541],
         [2.5300],
         [2.6547],
         [3.3686],
         [4.3545],
         [4.1732],
         [3.0626],
         [2.5300],
         [1.7254],
         [2.2014]]])
